In [26]:
import os
import subprocess
import time
import re

In [27]:
def run_iperf_test_and_print_table(scenario_id):
    cmd = [
        "sudo", "himage", f"pc3@{scenario_id}", "iperf", "-c", "10.0.4.20", "-n", "100M", "-P", "1", "-i", "1"
    ]

    print("=" * 56)
    print("Running TCP test between PC3 and PC4...")

    process = subprocess.run(cmd, capture_output=True, text=True)
    output = process.stdout

    pattern = re.compile(r"\[\s*\d+\]\s+([\d.]+-[\d.]+)\s+sec\s+([\d.]+)\s+MBytes\s+([\d.]+)\s+Mbits/sec")
    results = pattern.findall(output)

    if results:
        # Cabeçalho
        print("\n{:<15} {:<15} {:<15}".format("Interval", "Transfer (MB)", "Bandwidth (Mbps)"))
        print("-" * 45)

        # Linhas da tabela
        for interval, transfer, bandwidth in results:
            print("{:<15} {:<15} {:<15}".format(interval, transfer, bandwidth))

        print()
    else:
        print("❌ Nenhum resultado encontrado na saída do iperf.")
        print("Saída bruta:")
        print(output)


In [ ]:
def run_imunes_simulation(
    topology_file="scenario.imn",
    bandwidth="100000000",
    scenario_id="i2002",
    delay="10000",
    fluxes="1"
):
    log_dir = "./log"
    os.makedirs(log_dir, exist_ok=True)

    print("=" * 56)
    print(f"Starting IMUNES simulation with scenario ID: {scenario_id}")

    # Start simulation
    subprocess.run(
        ["sudo", "imunes", "-b", "-e", scenario_id, topology_file],
        stdout=subprocess.DEVNULL
    )

    time.sleep(5)

    print("=" * 56)
    print("Simulation started, applying commands...")

    # Link configurations
    links = [
        "router1:pc1",
        "router1:pc3",
        "router2:pc2",
        "router2:pc4",
        "router2:router1"
    ]
    for link in links:
        subprocess.run(
            ["sudo", "vlink", "-bw", bandwidth, "-dly", delay, f"{link}@{scenario_id}"],
            stdout=subprocess.DEVNULL
        )

    time.sleep(2)

    # Check status
    for link in links:
        subprocess.run(
            ["sudo", "vlink", "-s", f"{link}@{scenario_id}"]
        )

    time.sleep(2)

    # Start iperf servers
    subprocess.Popen(
        ["sudo", "himage", f"pc2@{scenario_id}", "iperf", "-s"],
        stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
    )
    subprocess.Popen(
        ["sudo", "himage", f"pc4@{scenario_id}", "iperf", "-s"],
        stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
    )

    # Background UDP traffic
    subprocess.Popen(
        ["sudo", "himage", f"pc1@{scenario_id}", "iperf", "-c", "10.0.3.20", "-u", "-t", "100000", "-b", "10M"],
        stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
    )

    time.sleep(2)

    # TCP test
    print("=" * 56)
    run_iperf_test_and_print_table(scenario_id)

    # Stop simulation
    print("=" * 56)
    print(f"Stopping IMUNES simulation with scenario ID: {scenario_id}")
    subprocess.run(["sudo", "imunes", "-b", "-e", scenario_id], stdout=subprocess.DEVNULL)
    print("=" * 56)
    print("Simulation stopped")



In [ ]:
run_imunes_simulation(
    topology_file="scenario.imn",
    bandwidth="100000000",
    scenario_id="i2006",
    delay="10000",
    fluxes="1"
)

Starting IMUNES simulation with scenario ID: i2005


Simulation started, applying commands...
pc1:router1@i2005:
	bandwidth 100Mbps
	delay 10ms
pc3:router1@i2005:
	bandwidth 100Mbps
	delay 10ms
router2:pc2@i2005:
	bandwidth 100Mbps
	delay 10ms
router2:pc4@i2005:
	bandwidth 100Mbps
	delay 10ms
router1:router2@i2005:
	bandwidth 100Mbps
	delay 10ms
Running TCP test between PC3 and PC4...
Running TCP test between PC3 and PC4...

Interval        Transfer (MB)   Bandwidth (Mbps)
---------------------------------------------
0.0000-1.0000   10.5            88.1           
1.0000-2.0000   10.9            91.2           
2.0000-3.0000   10.6            89.1           
3.0000-4.0000   10.5            88.1           
4.0000-5.0000   11.2            94.4           
5.0000-6.0000   11.5            96.5           
6.0000-7.0000   10.8            90.2           
7.0000-8.0000   10.6            89.1           
8.0000-9.0000   10.8            90.2           
9.0000-9.5937   2.62            37.1           
0.0000-9.5937   100             87.4           

